In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer as tfdV
from nltk.corpus import stopwords 
import pandas as pd
import nltk 
import re
splits = {'train': 'simplified/train-00000-of-00001.parquet', 'validation': 'simplified/validation-00000-of-00001.parquet', 'test': 'simplified/test-00000-of-00001.parquet'}
train  = pd.read_parquet("hf://datasets/google-research-datasets/go_emotions/" + splits["train"])
validation = pd.read_parquet("hf://datasets/google-research-datasets/go_emotions/" + splits["validation"])
test = pd.read_parquet("hf://datasets/google-research-datasets/go_emotions/" + splits["test"])

In [57]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/erdaulet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [64]:
train

,text,labels,id,cleaned_text
0,My favourite food is anything I didn't have to...,[27],eebbqej,favourite food anything didnt cook
1,"Now if he does off himself, everyone will thin...",[27],ed00q6i,everyone think hes laugh screwing people inste...
2,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj,fuck bayless isoing
3,To make her feel threatened,[14],ed7ypvh,make feel threatened
4,Dirty Southern Wankers,[3],ed0bdzj,dirty southern wankers
...,...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,[18],edsb738,added mate well ive got bow love hunting aspec...
43406,Always thought that was funny but is it a refe...,[6],ee7fdou,always thought funny reference anything
43407,What are you talking about? Anything bad that ...,[3],efgbhks,talking anything bad happened name fault good ...
43408,"More like a baptism, with sexy results!",[13],ed1naf8,like baptism sexy results


In [60]:
train['cleaned_text'] = train['text'].apply(preprocess)
validation['cleaned_text'] = validation['text'].apply(preprocess)
test['cleaned_text'] = test['text'].apply(preprocess)

In [61]:
X_train = train['cleaned_text']
X_val = validation['cleaned_text']
test = test['cleaned_text']

In [66]:
y_train = train['labels']
y_val = validation['labels']

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer() 
X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val)
X_test = vectorizer.transform(test)

In [105]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_train_binary = mlb.fit_transform(y_train)
y_val_binary = mlb.fit_transform(y_val)

In [111]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense,Input,Dropout
from keras import regularizers

model = Sequential([
    Dense(128, activation='relu', kernel_regularizer = regularizers.l2(0.001)),
    Dropout(0.3),
    Dense(len(mlb.classes_), activation='sigmoid')
])

model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['f1_score']
)

In [112]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    verbose=1,
    restore_best_weights=True,
    mode='max',
)

In [113]:
model.fit(X_train,y_train_binary,epochs=20,batch_size=32,validation_data=(X_val,y_val_binary),callbacks=[early_stop])

Epoch 1/20
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 33s 24ms/step - f1_score: 0.0216 - loss: 0.2684 - val_f1_score: 0.0175 - val_loss: 0.1507
Epoch 2/20
   4/1357 ━━━━━━━━━━━━━━━━━━━━ 28s 21ms/step - f1_score: 0.0191 - loss: 0.1487 

/Users/erdaulet/Desktop/allProjects/myenv/lib/python3.11/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: f1_score,loss,val_f1_score,val_loss
  current = self.get_monitor_value(logs)


 703/1357 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - f1_score: 0.0177 - loss: 0.1518

KeyboardInterrupt: 